In [1]:
import torch.utils.data as data
import torch
from PIL import Image
import os
import os.path
import numpy as np
from numpy.random import randint
import torchvision
from torchvision.transforms import *
import torch.nn as nn
import re
from torch.nn import functional as F
import time

import jpeg4py as jpeg
import matplotlib.pyplot as plt

In [83]:
class VideoRecord(object):
    def __init__(self, row):
        self._data = row

    @property
    def label(self):
        return int(self._data[0])

    @property
    def path(self):
        return self._data[1]

    @property
    def num_frames(self):
        return int(self._data[2])


class I3DDataSet(data.Dataset):
    def __init__(self, root_path, list_file, sample_frames=32,
                 image_tmpl='frame_{:06d}.jpg', transform=None,
                 force_grayscale=False, train_mode=True, test_clips=10, chunk_set=None):
        self.root_path = root_path
        self.list_file = list_file
        self.sample_frames = sample_frames
        self.image_tmpl = image_tmpl
        self.train_mode = train_mode
        if not self.train_mode:
            self.num_clips = test_clips

        if transform is not None:
            self.transform = transform
        else:
            self.transform = t.GroupToTensorStack()

        self._parse_list(chunk_set)

    def _load_image(self, video_dir, idx):
        img_path = os.path.join(self.root_path, video_dir, self.image_tmpl.format(idx))
        try:
            # Loading images with PIL is much slower!!
            return [Image.fromarray(jpeg.JPEG(img_path).decode())]
        except IOError:
            print("Couldn't load image:{}".format(img_path))
            return None

    def _parse_list(self, chunk_set):
        self.video_list = [VideoRecord(x.strip().split(' ')) for x in open(self.list_file)]
        if chunk_set is not None:
            self.video_list = self.video_list[chunk_set[0]:chunk_set[1]]

    def _sample_indices(self, record):
        """
        :param record: VideoRecord
        :return: list
        """
        expanded_sample_length = self.sample_frames * 4  # in order to drop every other frame
        if record.num_frames >= expanded_sample_length:
            start_pos = randint(record.num_frames - expanded_sample_length + 1)
            offsets = range(start_pos, start_pos + expanded_sample_length, 4)
        elif record.num_frames > self.sample_frames*2:
            start_pos = randint(record.num_frames - self.sample_frames*2 + 1)
            offsets = range(start_pos, start_pos + self.sample_frames*2, 2)
        elif record.num_frames > self.sample_frames:
            start_pos = randint(record.num_frames - self.sample_frames + 1)
            offsets = range(start_pos, start_pos + self.sample_frames, 1)
        else:
            offsets = np.sort(randint(record.num_frames, size=self.sample_frames))

        offsets = [int(v) for v in offsets]
        return offsets

    def _get_test_indices(self, record):
        tick = (record.num_frames - self.sample_frames*2 + 1) / float(self.num_clips)
        sample_start_pos = np.array([int(tick * x) for x in range(self.num_clips)])
        offsets = []
        for p in sample_start_pos:
            offsets.extend(range(p, p+self.sample_frames*2, 2))

        checked_offsets = []
        for f in offsets:
            new_f = int(f)
            if new_f < 0:
                new_f = 0
            elif new_f >= record.num_frames:
                new_f = record.num_frames - 1
            checked_offsets.append(new_f)

        return checked_offsets

    def __getitem__(self, index):
        t1 = time.time()
        record = self.video_list[index]

        if self.train_mode:
            segment_indices = self._sample_indices(record)
            process_data, label = self.get(record, segment_indices)
            while process_data is None:
                index = randint(0, len(self.video_list) - 1)
                process_data, label = self.__getitem__(index)
        else:
            segment_indices = self._get_test_indices(record)
            process_data, label = self.get(record, segment_indices)
            if process_data is None:
                raise ValueError('sample indices:', record.path, segment_indices)
        print('Getting frames took: {:.3} s'.format(time.time() - t1))
        return process_data, label

    def get(self, record, indices):
        uniq_imgs = {}
        uniq_id = np.unique(indices)
        for ind in uniq_id:
            seg_img = self._load_image(record.path, ind)

            if seg_img is None:
                return None, None
            uniq_imgs[ind] = seg_img

        images = [uniq_imgs[i][0] for i in indices]
        images = self.transform(images)
        return images, record.label

    def __len__(self):
        return len(self.video_list)


In [32]:
import os

import cv2
import jpeg4py as jpeg
import numpy as np
import torch.utils.data as data
from numpy.random import randint
from PIL import Image


class VideoRecord2(object):
    def __init__(self, video_path):
        self.path = video_path
        self.video = cv2.VideoCapture(self.path)
        self.num_frames = self.get_num_frames()
        
    def get_num_frames(self):
        count = -1
        ret = True
        while(ret):
            ret, frame = self.video.read()
            count += 1
        self.video.set(2, 0)
        return count

    def get_frames(self, indices):
        """
        Argument:
            indices : sorted list of frames indices
        Returns:
            images : Dictionary in format {frame_id: PIL Image}
        """
        images = dict()
        for count in range(min(indices), max(indices)+1):
            ret, frame = self.video.read()
            if ret is False:
                print('Could not load frame {} from video {}'.format(count, self.path))
                return None
            
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if count in indices:
                images[count] = Image.fromarray(frame)
        self.video.release()
        return images


class VideoDataset(data.Dataset):
    def __init__(self, root_path, list_file, sample_frames=32,
                 image_tmpl='frame_{:06d}.jpg', transform=None,
                 train_mode=True, test_clips=10, num_channels=3):
        self.root_path = root_path
        self.list_file = list_file
        self.sample_frames = sample_frames
        self.num_channels = num_channels
        self.image_tmpl = image_tmpl
        self.transform = transform
        self.train_mode = train_mode
        if not self.train_mode:
            self.num_clips = test_clips

        self._parse_list()

    def _load_image(self, video_dir, idx):
        img_path = os.path.join(self.root_path, video_dir, self.image_tmpl.format(idx))
        try:
            # Loading images with PIL is slower.
            return [Image.fromarray(jpeg.JPEG(img_path).decode()).convert('RGB')]
        except IOError:
            print("Couldn't load image:{}".format(img_path))
            return None

    def _parse_list(self):
        self.video_list = [x.strip().split(' ') for x in open(self.list_file)]

    def _sample_indices(self, record):
        """
        :param record: VideoRecord
        :return: list
        """
        expanded_sample_length = self.sample_frames * 4  # in order to drop every other frame
        if record.num_frames >= expanded_sample_length:
            start_pos = randint(record.num_frames - expanded_sample_length + 1)
            offsets = range(start_pos, start_pos + expanded_sample_length, 4)
        elif record.num_frames > self.sample_frames*2:
            start_pos = randint(record.num_frames - self.sample_frames*2 + 1)
            offsets = range(start_pos, start_pos + self.sample_frames*2, 2)
        elif record.num_frames > self.sample_frames:
            start_pos = randint(record.num_frames - self.sample_frames + 1)
            offsets = range(start_pos, start_pos + self.sample_frames, 1)
        else:
            offsets = np.sort(randint(record.num_frames, size=self.sample_frames))

        offsets = [int(v) for v in offsets]
        return offsets

    def _get_test_indices(self, record):
        tick = (record.num_frames - self.sample_frames*2 + 1) / float(self.num_clips)
        sample_start_pos = np.array([int(tick * x) for x in range(self.num_clips)])
        offsets = []
        for p in sample_start_pos:
            offsets.extend(range(p, p+self.sample_frames*2, 2))

        checked_offsets = []
        for f in offsets:
            new_f = int(f)
            if new_f < 0:
                new_f = 0
            elif new_f >= record.num_frames:
                new_f = record.num_frames - 1
            checked_offsets.append(new_f)

        return checked_offsets

    def __getitem__(self, index):
        t1 = time.time()
        label, video_path = self.video_list[index]
        record = VideoRecord2(os.path.join(self.root_path, video_path))

        if self.train_mode:
            segment_indices = self._sample_indices(record)
            process_data = self.get(record, segment_indices)
            while process_data is None:
                index = randint(0, len(self.video_list) - 1)
                process_data, label = self.__getitem__(index)
        else:
            segment_indices = self._get_test_indices(record)
            process_data = self.get(record, segment_indices)
            if process_data is None:
                raise ValueError
        print('Getting frames took: {:.3} s'.format(time.time() - t1))
        
        data = process_data.squeeze(0)
        data = data.view(self.num_channels, -1, self.sample_frames, data.size(2), data.size(3)).contiguous()
        data = data.permute(1, 0, 2, 3, 4).contiguous()
        
        return data, int(label)

    def get(self, record, indices):
        uniq_imgs = {}
        uniq_id = np.unique(indices)
        uniq_imgs = record.get_frames(uniq_id)
        
        if uniq_imgs is None:
            return None

        images = [uniq_imgs[i] for i in indices]
        images = self.transform(images)
        return images

    def __len__(self):
        return len(self.video_list)


In [33]:
class GroupCenterCrop(object):

    def __init__(self, size):
        self.worker = torchvision.transforms.CenterCrop(size)

    def __call__(self, img_group):
        return [self.worker(img) for img in img_group]
    
    
class GroupNormalize(object):
    def __init__(self, mean=None, std=None, num_channels=3):
        self.mean = mean if mean is not None else [0] * num_channels
        self.std = std if std is not None else [1] * num_channels

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.sub_(m).div_(s)
            
        return tensor
    

class GroupResize(object):

    def __init__(self, size, interpolation=Image.BILINEAR):
        self.worker = torchvision.transforms.Resize(size, interpolation)

    def __call__(self, img_group):
        return [self.worker(img) for img in img_group]
    
    
class GroupToTensorStack(object):

    def __call__(self, img_group):
        return torch.stack([torchvision.transforms.ToTensor()(img) for img in img_group], dim=1)


In [34]:
test_list = '/media/v-pakova/New Volume1/Datasets/Kinetics/400/val_clips_256_list.txt'

input_mean = [0.5, 0.1, 0.9]
input_std = [0.225, 0.1, 0.5]

data_loader = torch.utils.data.DataLoader(
        VideoDataset("/media/v-pakova/New Volume1/Datasets/Kinetics/400/val_clips_256", 
                   test_list, 
                   image_tmpl="frame_{:06d}.jpg",
                   train_mode=False,
                   transform=torchvision.transforms.Compose([
                       GroupResize(256),
                       GroupToTensorStack(),
#                        GroupNormalize(mean=input_mean),
                    ])),
        batch_size=1, shuffle=False) #, pin_memory=True)


In [35]:
video_data_gen = enumerate(data_loader)
for i, (mydata, label) in video_data_gen:
    print((i, mydata.shape, label))
    video_data = mydata
#     if i > 3:
    break

Getting frames took: 8.47 s
(0, torch.Size([1, 10, 3, 32, 256, 340]), tensor([0]))


In [88]:
test_list = '/media/v-pakova/New Volume/Datasets/Kinetics/400/val_list.txt'

input_mean = [0.5, 0.1, 0.9]
input_std = [0.225, 0.1, 0.5]

data_loader = torch.utils.data.DataLoader(
        I3DDataSet("/media/v-pakova/New Volume/Datasets/Kinetics/400/val_frames_256", 
                   test_list, 
                   image_tmpl="frame_{:06d}.jpg",
                   train_mode=False,
                   transform=torchvision.transforms.Compose([
                       GroupResize(256),
                       GroupToTensorStack(),
#                        GroupNormalize(mean=input_mean),
                    ])),
        batch_size=1, shuffle=False) #, pin_memory=True)


In [89]:
frame_data_gen = enumerate(data_loader)
for i, (mydata, label) in frame_data_gen:
    print((i, mydata.shape, label))
    frame_data = mydata
    if i > 4:
        break

Getting frames took: 2.43 s
(0, torch.Size([1, 3, 320, 256, 340]), tensor([0]))
Getting frames took: 2.25 s
(1, torch.Size([1, 3, 320, 256, 454]), tensor([0]))
Getting frames took: 2.66 s
(2, torch.Size([1, 3, 320, 256, 340]), tensor([0]))
Getting frames took: 3.32 s
(3, torch.Size([1, 3, 320, 256, 454]), tensor([0]))
Getting frames took: 2.04 s
(4, torch.Size([1, 3, 320, 256, 340]), tensor([0]))
Getting frames took: 1.99 s
(5, torch.Size([1, 3, 320, 256, 454]), tensor([0]))


In [94]:
print(torch.allclose(frame_data, video_data))
print(frame_data == video_data)

False
tensor([[[[[0, 1, 0,  ..., 1, 0, 0],
           [0, 1, 0,  ..., 0, 0, 0],
           [0, 1, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 1, 1],
           [0, 0, 0,  ..., 0, 1, 1],
           [0, 0, 0,  ..., 0, 1, 1]],

          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 1,  ..., 1, 1, 1],
           [0, 0, 0,  ..., 1, 1, 1],
           [0, 1, 1,  ..., 1, 1, 1]],

          [[0, 1, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 1],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 1, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 1],
           [0, 0, 0,  ..., 0, 0, 1]],

          ...,

          [[1, 1, 1,  ..., 0, 0, 0],
           [1, 1, 1,  ..., 0, 0, 0],
           [1, 1, 1,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 1, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],

          [[0, 0, 

In [1]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, time_kernel=1, space_stride=1, downsample=None, addnon=False):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, 
                               planes, 
                               kernel_size=(time_kernel,1,1), 
                               padding=(int((time_kernel-1)/2), 0,0),
                               bias=False) # timepadding: make sure time-dim not reduce
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(planes, 
                               planes, 
                               kernel_size=(1,3,3), 
                               stride=(1,space_stride,space_stride),
                               padding=(0,1,1), 
                               bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, 
                               planes * 4, 
                               kernel_size=(1,1,1), 
                               bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.addnon = addnon
        if self.addnon:
            self.nonlocal_block = NonLocalBlock3D(in_channels=planes * 4, mode='embedded_gaussian')

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        if self.addnon:
            out = self.nonlocal_block(out)
        return out



class I3DResNet(nn.Module):

    def __init__(self, block, layers, frame_num=32, num_classes=400):
        if torch.cuda.is_available():
            torch.set_default_tensor_type('torch.cuda.FloatTensor')
        self.inplanes = 64
        super(I3DResNet, self).__init__()
        self.conv1 = nn.Conv3d(3, 64, 
                               kernel_size=(5,7,7), 
                               stride=(2,2,2), 
                               padding=(2,3,3),
                               bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3,3,3), stride=(2,2,2), padding=(1,1,1))
        self.layer1 = self._make_layer_inflat(block, 64, layers[0], first_block=True)
        self.temporalpool = nn.MaxPool3d(kernel_size=(3,1,1), stride=(2,1,1), padding=(1,0,0))
        self.layer2 = self._make_layer_inflat(block, 128, layers[1], space_stride=2)
        self.layer3 = self._make_layer_inflat(block, 256, layers[2], space_stride=2)
        self.layer4 = self._make_layer_inflat(block, 512, layers[3], space_stride=2)
        self.avgpool = nn.AvgPool3d((int(frame_num/8),7,7))
        self.avgdrop =nn.Dropout(0.5)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

   
    def _make_layer_inflat(self, block, planes, blocks, space_stride=1, first_block=False):
        downsample = None
        if space_stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(self.inplanes, planes * block.expansion,
                          kernel_size=(1,1,1), stride=(1,space_stride,space_stride), bias=False),
                nn.BatchNorm3d(planes * block.expansion),
            )

        layers = []
        if blocks%2 == 0 or first_block:
            time_kernel = 3
        else:
            time_kernel = 1
            
        # Add first block
        layers.append(block(self.inplanes, planes, time_kernel, space_stride, downsample, addnon = False))
        self.inplanes = planes * block.expansion
        
        if first_block:
            for i in range(1, blocks):
                layers.append(block(self.inplanes, planes, time_kernel))
        elif blocks%2 == 0:
            time_kernel = 1
            add_nonlocal = True
            for i in range(1, blocks):
                layers.append(block(self.inplanes, planes, time_kernel, addnon=add_nonlocal))
                time_kernel = (time_kernel + 2)%4
                add_nonlocal = not add_nonlocal
        else:
            time_kernel = 3
            for i in range(1, blocks):
                layers.append(block(self.inplanes, planes, time_kernel))
                time_kernel = (time_kernel + 2)%4
         
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.temporalpool(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.permute(0, 2, 1, 3, 4).contiguous()
        x = x.view(x.size(0), -1)
        x = self.avgdrop(x)
        x = self.fc(x)

        return x


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = I3DResNet(Bottleneck, [3, 4, 6, 3], **kwargs)

    return model



class _NonLocalBlockND(nn.Module):
    def __init__(self, in_channels, inter_channels=None, dimension=3, mode='embedded_gaussian',
                 sub_sample=True, bn_layer=True):
        super(_NonLocalBlockND, self).__init__()

        assert dimension in [1, 2, 3]
        assert mode in ['embedded_gaussian', 'gaussian', 'dot_product', 'concatenation']

        # print('Dimension: %d, mode: %s' % (dimension, mode))

        self.mode = mode
        self.dimension = dimension
        self.sub_sample = sub_sample

        self.in_channels = in_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
            if self.inter_channels == 0:
                self.inter_channels = 1

        if dimension == 3:
            conv_nd = nn.Conv3d
            max_pool = nn.MaxPool3d
            bn = nn.BatchNorm3d
        elif dimension == 2:
            conv_nd = nn.Conv2d
            max_pool = nn.MaxPool2d
            bn = nn.BatchNorm2d
        else:
            conv_nd = nn.Conv1d
            max_pool = nn.MaxPool1d
            bn = nn.BatchNorm1d

        self.g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                         kernel_size=1, stride=1, padding=0)       
        nn.init.kaiming_normal_(self.g.weight)
        nn.init.constant_(self.g.bias,0)
        
        if bn_layer:
            self.W = nn.Sequential(
                conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                        kernel_size=1, stride=1, padding=0),
                bn(self.in_channels)
            )
            nn.init.kaiming_normal_(self.W[0].weight)
            nn.init.constant_(self.W[0].bias, 0)
            nn.init.constant_(self.W[1].weight, 0)
            nn.init.constant_(self.W[1].bias, 0)

            
        else:
            self.W = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                             kernel_size=1, stride=1, padding=0)
            nn.init.kaiming_normal(self.W.weight)
            nn.init.constant(self.W.bias, 0)

        self.theta = None
        self.phi = None

        self.theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=1, stride=1, padding=0)
        self.phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0)
        
        if self.mode == "embedded_gaussian":
            self.operation_function = self._embedded_gaussian

        if sub_sample:
            self.g = nn.Sequential(self.g, max_pool(kernel_size=2))
            if self.phi is None:
                self.phi = max_pool(kernel_size=2)
            else:
                self.phi = nn.Sequential(self.phi, max_pool(kernel_size=2))

    def forward(self, x):
        '''
        :param x: (b, c, t, h, w)
        :return:
        '''

        output = self.operation_function(x)
        return output

    def _embedded_gaussian(self, x):
        batch_size = x.size(0)

        # g=>(b, c, t, h, w)->(b, 0.5c, t, h, w)->(b, thw, 0.5c)
        g_x = self.g(x).view(batch_size, self.inter_channels, -1)
        g_x = g_x.permute(0, 2, 1)

        # theta=>(b, c, t, h, w)[->(b, 0.5c, t, h, w)]->(b, thw, 0.5c)
        # phi  =>(b, c, t, h, w)[->(b, 0.5c, t, h, w)]->(b, 0.5c, thw)
        # f=>(b, thw, 0.5c)dot(b, 0.5c, twh) = (b, thw, thw)
        theta_x = self.theta(x).view(batch_size, self.inter_channels, -1)
        theta_x = theta_x.permute(0, 2, 1)
        phi_x = self.phi(x).view(batch_size, self.inter_channels, -1)
        f = torch.matmul(theta_x, phi_x)
        f_div_C = F.softmax(f, dim=-1)

        # (b, thw, thw)dot(b, thw, 0.5c) = (b, thw, 0.5c)->(b, 0.5c, t, h, w)->(b, c, t, h, w)
        y = torch.matmul(f_div_C, g_x)
        y = y.permute(0, 2, 1).contiguous()
        y = y.view(batch_size, self.inter_channels, *x.size()[2:])
        W_y = self.W(y)
        z = W_y + x

        return z

class NonLocalBlock3D(_NonLocalBlockND):
    def __init__(self, in_channels, inter_channels=None, mode='embedded_gaussian', sub_sample=True, bn_layer=True):
        super(NonLocalBlock3D, self).__init__(in_channels,
                                              inter_channels=inter_channels,
                                              dimension=3, mode=mode,
                                              sub_sample=sub_sample,
                                              bn_layer=bn_layer)

NameError: name 'nn' is not defined

In [2]:
def copy_weigths_i3dResNet(weigths_file_path, new_model, save_model=False, new_model_name=None):
    with open(weigths_file_path, 'rb') as model_file:
        pretrained_data = pickle.load(model_file, encoding='latin1')

    pretrained_data1 = pretrained_data['blobs']
    # Removing training parameters
    pretrained_data2 = {k: v for k, v in pretrained_data1.items() if
                        ('momentum' not in k) and ('bn_rm' not in k) and ('bn_riv' not in k)
                        and ('lr' not in k) and ('model_iter' not in k)}

    pretrained_data_list = sorted(pretrained_data2.items())

    # Renaming layers
    pretrained_data3 = {}
    for k, v in pretrained_data_list:
        a = k[:]
        # Correcting the end
        if a[-2:] == '_b':
            a = a[:-2]+'.bias'
        elif a[-2:] == '_s' or a[-2:] == '_w':
            a = a[:-2]+'.weight'

        # Correcting the begin
        a = a.replace('res_conv1_bn', 'bn1')
        a = a.replace('pred', 'fc')
        r = re.compile('res._*')
        if r.match(a) is not None:
            layer = int(a[3])-1
            a = 'layer'+str(layer)+'.'+a[5:]

        # Correcting the middle
        a = a.replace('_branch1_bn', '.downsample.1')
        a = a.replace('_branch1', '.downsample.0')

        for i, l in enumerate(['a', 'b', 'c']):
            a = a.replace('_branch2{}_bn'.format(l), '.bn{}'.format(i+1))
            a = a.replace('_branch2{}'.format(l), '.conv{}'.format(i+1))

        # Correcting nonlocal
        if 'nonlocal' in a:
            layer = int(a[13])-1
            sub_layer = a[15]
            a = 'layer{}.{}.nonlocal_block.'.format(layer, sub_layer) + a[17:]
            a = a.replace('out', 'W.0')
            a = a.replace('bn', 'W.1')
            a = a.replace('phi', 'phi.0')
            a = a.replace('.g.', '.g.0.')

        pretrained_data3[a] = {'old_name': k, 'data': v}

    # Checking name, shape and number of layers
    param_i3d_keys = new_model.state_dict().keys()
    count = 0
    for k, v in pretrained_data3.items():
        assert(k in param_i3d_keys)
        count += 1
        assert(new_model.state_dict()[k].shape == v['data'].shape)

    assert count == len(list(new_model.named_parameters()))

    # Copying weigths
    for k, v in new_model.named_parameters():
        new_data = pretrained_data3[k]['data']
        v.data = nn.Parameter(torch.Tensor(new_data))

    return new_model


In [3]:
blank_i3d = resnet50()
loaded_model = torch.load('../resnet50_i3d_pre_trained.pt')
# pre_trained_file = ("/media/v-pakova/New Volume/OnlineActionRecognition/models/pre-trained/"
#                         "non-local/i3d_nonlocal_32x2_IN_pretrain_400k.pkl")
# resnet_i3d = copy_weigths_i3dResNet(pre_trained_file, blank_i3d)

NameError: name 'resnet50' is not defined

In [10]:
for i, (mydata, label) in data_gen:
#     print((i, mydata.shape, label))
    d1 = mydata.squeeze(0)
    d2 = d1.view(3, -1, 32, d1.size(2), d1.size(3)).contiguous()
    d3 = d2.permute(1,0,2,3,4).contiguous()
    
    with torch.no_grad():
        input_var = torch.Tensor(d3.cuda())

    #     r1 = blank_i3d(input_var).data.cpu().numpy().copy()
    #     r2 = loaded_model(input_var).data.cpu().numpy().copy()
        r3 = resnet_i3d(input_var).data.cpu().numpy().copy()

    #     r11 = r1.reshape((10*1, 400)).mean(axis=0).reshape((1, 400))
    #     r22 = r2.reshape((10*1, 400)).mean(axis=0).reshape((1, 400))
        r33 = r3.reshape((10*1, 400)).mean(axis=0).reshape((1, 400))

        print(np.argmax(r33))#, np.argmax(r22), np.argmax(r33))

278
0
67
0
0
0


KeyboardInterrupt: 

In [10]:
print('Compare blank with loaded')
for p1, p2 in zip(blank_i3d.parameters(), loaded_model.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        print(False)
print(True)

print('Compare blank with copied')
for p1, p2 in zip(blank_i3d.parameters(), resnet_i3d.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        print(False)
print(True)

print('Compare loaded with copied')
for p1, p2 in zip(resnet_i3d.parameters(), loaded_model.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        print(False)
print(True)

Compare blank with loaded
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fa

NameError: name 'resnet_i3d' is not defined